# ECC058 - DHLab

# Worked example: URLImage Class and IIIF


#### Prof. Stephen White
#### stephen.white@unive.it


#### Resources:
https://iiif.io/api/image/2.0/#identifier

**BaseURL/Region/Size/Rotation/ImageFormat**

# Display Image from Inscription Image URL for Query Results
1. Load query results page
2. Find a single result
    - Create BeautifulSoup from result string
    - Get a list of all results reffer 'table' elements of class 'treffertabelle'
    - Load a single result into a variable
3. Find and load iiif image page URL link if exist
    - Find iiif imgage page link to inscription image
    - If exist load href property into a variable as full URL
    - Load URL results
4. Find and load URL for IIIF json manifest
    - Create BeautifulSoup from result string
    - Get a list of all results reffer 'table' elements of class 'treffertabelle'
    - Load a single result into a dictionary variable
    - Using the manifest, find and load into a varible the IIIF image url
5. Display Image
    - Create URLImage object
    - Load Image
    - Show Image

In [ ]:
#developement commands
%reload_ext autoreload
%autoreload 2


#load needed libraries
import urllib.request
from bs4 import BeautifulSoup
import re,json
import pandas as pd
import EpiProject.urlimage as urlimage

In [ ]:
# Create query variables
# base url string of site
urlBase = "https://edh-www.adw.uni-heidelberg.de"
# query parameters for the url, page size set to 100
urlQueryBase = "/inschrift/erweiterteSuche?hd_nr=&tm_nr=&beleg=c&land=&fo_antik=&fo_modern=&fundstelle=&region=&compFundjahr=eq&fundjahr=&aufbewahrung=&inschriftgattung=&sprache=L&inschrifttraeger=&compHoehe=eq&hoehe=&compBreite=eq&breite=&compTiefe=eq&tiefe=&bh=&palSchreibtechnik=&dat_tag=&dat_monat=&dat_jahr_a=&dat_jahr_e=&hist_periode=&religion=&literatur=&kommentar=&p_name=&p_praenomen=&p_nomen=&p_cognomen=&p_supernomen=&p_tribus=&p_origo=&p_geschlecht=&p_status=&compJahre=eq&p_lJahre=&compMonate=eq&p_lMonate=&compTage=eq&p_lTage=&compStunden=eq&p_lStunden=&atext1=&bool=AND&atext2=&beleg89=ja&nurMitFoto=ja&sort=hd_nr&anzahl=100&addFeldMaterial=ja&addFeldDTyp=ja&addFeldIGat=ja&start="
# offset of the query
offset = 0
# parameter to specify language as english
paramLang = "&lang=en"

#create query
url = urlBase + urlQueryBase + str(offset)
#print(url)

In [ ]:
#1. Load query results page
f = urllib.request.urlopen(url)
htmlDocString = f.read() 
f.close()
# take a quick look at the html
#print(htmlDocString)


In [ ]:
#2. Find a single result

#    - Create BeautifulSoup from result string
htmlSoup = BeautifulSoup(htmlDocString,'html.parser')

#print(htmlSoup.prettify())

#    - Get a list of all results reffer 'table' elements of class 'treffertabelle'
tableRefferList = htmlSoup.select('table.treffertabelle')


#print(type(tableRefferList)) #<class 'list'>
#print(len(tableRefferList))  #100

#    - Load a single result into a variable
#set curTable to 3rd table scrap info for a single result while developing
#later this will just be a loop variable
curTable = tableRefferList[2]


print (curTable.prettify())

# Play Around Area - Scratch Code